In [37]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, QuantumRegister, ClassicalRegister, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2023-02-28 00:46:15,174: Credentials are already in use. The existing account in the session will be replaced.


In [55]:
def compare_nums(A, B):
    ''' This function uses the CSWAP test, which measures the overlap of two quantum states'''
    # Convert ints to binary strings
    A = bin(A)[2:]
    B = bin(B)[2:]
    # Make both strings the same length by changing preceding 0s
    if len(A) > len(B):
        B = '0'*(len(A)-len(B))+B
    else:
        A = '0'*(len(B)-len(A))+A
    
    # Create quantum registers for each number, including an ancillary qubit to implement the test
    qr_A = QuantumRegister(len(A), name="qr_A")
    qr_B = QuantumRegister(len(B), name="qr_B")
    qr_anc = QuantumRegister(1, name="ancillary")
    cr_measure = ClassicalRegister(1, name="measurement")
    qc = QuantumCircuit(qr_anc, qr_A, qr_B, cr_measure)
    
    # Set each quantum register to the corresponding quantum state
    for i in range(len(A)):
        if A[i] == '1':
            qc.x(qr_A[i])  # Puts qubit in the |1> state if the bitstring has a '1' there
        if B[i] == '1':
            qc.x(qr_B[i])
    
    qc.h(qr_anc[0])  # Ancillary qubit in superposition
    for i in range(len(A)):
        qc.cswap(qr_anc[0], qr_A[i], qr_B[i])  # Swap each qubit with the corresponding qubit in the other register
    qc.h(qr_anc[0])  # Phase kickback leads to the initial state changing
    qc.measure(qr_anc[0], cr_measure[0])  # Measurement
    
    # Run Experiment
    aer_sim = Aer.get_backend('aer_simulator')
    shots = 2048
    t_qpe = transpile(qc, aer_sim)
    qobj = assemble(t_qpe, shots=shots)
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    
    '''
    If the two states have no overlap |0> will be measured ~50% of the time
    For this problem, there will only ever be very high amounts of overlap (both states are equal) or no overlap (states are not equal),
    Thus, threshold for success is very high.
    '''
    
    if counts['0'] > shots * 0.99:  
        return True
    return False

In [75]:
def is_rectangle(A, B, C, D):
    sides = [A, B, C, D]
    two_equal = False
    for i in range(1, len(sides)):
        if compare_nums(A, sides[i]):
            two_equal = True
            del sides[i]
            del sides[0]
            break
    if two_equal:
        return compare_nums(sides[0], sides[1])
    else:
        return False

In [83]:
print(is_rectangle(1452,764,764,1452))

True
